In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import col
import pyspark.sql.types as t
import pyspark.sql.functions as f
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
hack_license = "hack_license"
dropoff_datetime = "dropoff_datetime"
dropoff_latitude = "dropoff_latitude"
dropoff_longitude = "dropoff_longitude"
fare_amount = "fare_amount"
medallion = "medallion"
mta_tax = "mta_tax"
passenger_count = "passenger_count"
payment_type = "payment_type"
pickup_datetime = "pickup_datetime"
pickup_latitude = "pickup_latitude"
pickup_longitude = "pickup_longitude"
rate_code = "rate_code"
store_and_fwd_flag = "store_and_fwd_flag"
surcharge = "surcharge"
tip_amount = "tip_amount"
tolls_amount = "tolls_amount"
total_amount = "total_amount"
trip_distance = "trip_distance"
trip_time_in_secs = "trip_time_in_secs"
vendor_id = "vendor_id"

In [ ]:
path_trip = '/content/drive/Shareddrives/Big Data/trip_data'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class TripWorker:
    trip_schema = t.StructType([
        t.StructField(medallion, t.StringType(), False),
        t.StructField(hack_license, t.StringType(), False),
        t.StructField(vendor_id, t.StringType(), False),
        t.StructField(rate_code, t.IntegerType(), False),
        t.StructField(store_and_fwd_flag, t.StringType(), False),
        t.StructField(pickup_datetime, t.TimestampType(), False),
        t.StructField(dropoff_datetime, t.TimestampType(), False),
        t.StructField(passenger_count, t.IntegerType(), False),
        t.StructField(trip_time_in_secs, t.IntegerType(), False),
        t.StructField(trip_distance, t.FloatType(), False),
        t.StructField(pickup_longitude, t.FloatType(), False),
        t.StructField(pickup_latitude, t.FloatType(), False),
        t.StructField(dropoff_longitude, t.FloatType(), False),
        t.StructField(dropoff_latitude, t.FloatType(), False),
    ])

    def __init__(self, spark: SparkSession):
        self.df: DataFrame = (spark.read
                              .csv(path_trip, self.trip_schema, header=True, ignoreLeadingWhiteSpace=True,
                                   ignoreTrailingWhiteSpace=True, nullValue='null'))

    def show(self, n=None):
        self.df.show(n=n)

    def count(self) -> int:
        return self.df.count()

    def schema(self):
        self.df.printSchema()


In [ ]:
def init_spark() -> SparkSession:
    return (SparkSession.builder
            .master("local")
            .appName("Project")
            .config(conf=SparkConf())
            .getOrCreate())

In [ ]:
spark = init_spark()

In [ ]:
tripWorker = TripWorker(spark)

In [ ]:
tripWorker.df.show()

+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|FE7B354FEB67B9C94...|2C78614ADC9C602EC...|      CMT|        1|                 N|2013-03-01 00:00:04|2013-03-01 00:19:03|              1|             1138|         14.3|        -73.7767|      40.645164|       -73.913925|       40.772614|
|B1370E6501ED61CED...|08BF18740B38D6472...| 

In [ ]:
tripWorker.schema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)



In [ ]:
num_cols = [f.name for f in tripWorker.df.schema.fields if isinstance(f.dataType, t.NumericType)]
tripWorker.df.describe(num_cols).show()

+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|         rate_code|   passenger_count|trip_time_in_secs|    trip_distance|  pickup_longitude|  pickup_latitude| dropoff_longitude|  dropoff_latitude|
+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|         173179759|         173179759|        173179759|        173179759|         173179759|        173179759|         173176321|         173176321|
|   mean|1.0337213022683558|1.7094193669596225|811.9999416964196|8.305137246352254|-72.45156327315385|39.75292675450606|-72.40810490722372|39.729983492483974|
| stddev|0.3391333355137789|1.3767439366809253|15815.40566502589|6639.903068837679|11.291790226013376|8.780317442469093|11.417103659618059|  8.86756258675792|
|    min|                 0|                 0

In [ ]:
from pyspark.sql.functions import col

class TripWorker:
    trip_schema = t.StructType([
        t.StructField(medallion, t.StringType(), False),
        t.StructField(hack_license, t.StringType(), False),
        t.StructField(vendor_id, t.StringType(), False),
        t.StructField(rate_code, t.IntegerType(), False),
        t.StructField(store_and_fwd_flag, t.StringType(), False),
        t.StructField(pickup_datetime, t.TimestampType(), False),
        t.StructField(dropoff_datetime, t.TimestampType(), False),
        t.StructField(passenger_count, t.IntegerType(), False),
        t.StructField(trip_time_in_secs, t.IntegerType(), False),
        t.StructField(trip_distance, t.FloatType(), False),
        t.StructField(pickup_longitude, t.FloatType(), False),
        t.StructField(pickup_latitude, t.FloatType(), False),
        t.StructField(dropoff_longitude, t.FloatType(), False),
        t.StructField(dropoff_latitude, t.FloatType(), False),
    ])

    preprocess_filter_condition = (
            (col(passenger_count) > 0)
            & (col(passenger_count) <= 9)
            & (col(trip_distance) > 0)
            & (col(trip_distance) < 70_000)
            & (col(trip_time_in_secs) > 0)
            & (col(trip_time_in_secs) < 60_000))

    def __init__(self, spark: SparkSession):
        self.df: DataFrame = (spark.read
                              .csv(path_trip, self.trip_schema, header=True, ignoreLeadingWhiteSpace=True,
                                   ignoreTrailingWhiteSpace=True, nullValue='null')
                              .filter(self.preprocess_filter_condition))


    def show(self, n=None):
        self.df.show(n=n)

    def count(self) -> int:
        return self.df.count()

    def schema(self):
        self.df.printSchema()

In [ ]:
tripWorker = TripWorker(spark)

In [ ]:
tripWorker.df.show()

+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|FE7B354FEB67B9C94...|2C78614ADC9C602EC...|      CMT|        1|                 N|2013-03-01 00:00:04|2013-03-01 00:19:03|              1|             1138|         14.3|        -73.7767|      40.645164|       -73.913925|       40.772614|
|B1370E6501ED61CED...|08BF18740B38D6472...| 

In [ ]:
num_cols = [f.name for f in tripWorker.df.schema.fields if isinstance(f.dataType, t.NumericType)]
tripWorker.df.describe(num_cols).show()

+-------+-------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|          rate_code|   passenger_count|trip_time_in_secs|     trip_distance|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|
+-------+-------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|          171899205|         171899205|        171899205|         171899205|         171899205|         171899205|         171899205|         171899205|
|   mean| 1.0270787523421065|1.7116061647870913|757.7813952775407|2.9229047508589625|-72.55375877214988|39.808676497132566|-72.55916906300425|39.812565326362915|
| stddev|0.30057568678750834|1.3771786361954093|560.5441541100786|20.957479508723758|10.967169280209196| 8.655816513663163|10.938848778547413| 8.685174083633767|
|    min|                  0

In [ ]:
path_fare = '/content/drive/Shareddrives/Big Data/trip_fare'

In [ ]:
class FareWorker:
    fare_schema = t.StructType([
        t.StructField(medallion, t.StringType(), False),
        t.StructField(hack_license, t.StringType(), False),
        t.StructField(vendor_id, t.StringType(), False),
        t.StructField(pickup_datetime, t.TimestampType(), False),
        t.StructField(payment_type, t.StringType(), False),
        t.StructField(fare_amount, t.FloatType(), False),
        t.StructField(surcharge, t.FloatType(), False),
        t.StructField(mta_tax, t.FloatType(), False),
        t.StructField(tip_amount, t.FloatType(), False),
        t.StructField(tolls_amount, t.FloatType(), False),
        t.StructField(total_amount, t.FloatType(), False),
    ])

    def __init__(self, spark: SparkSession):
        self.df = (spark.read
                   .csv(path_fare, self.fare_schema, header=True, ignoreTrailingWhiteSpace=True,
                        ignoreLeadingWhiteSpace=True, nullValue='null'))

    def show(self):
        self.df.show()

    def count(self) -> int:
        return self.df.count()

    def schema(self):
        self.df.printSchema()

In [ ]:
fareWorker = FareWorker(spark)

In [ ]:
fareWorker.df.show()

+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+------------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|      CMT|2013-01-01 15:11:48|         CSH|        6.5|      0.0|    0.5|       0.0|         0.0|         7.0|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...|      CMT|2013-01-06 00:18:35|         CSH|        6.0|      0.5|    0.5|       0.0|         0.0|         7.0|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...|      CMT|2013-01-05 18:49:41|         CSH|        5.5|      1.0|    0.5|       0.0|         0.0|         7.0|
|DFD2202EE08F7A8DC...|51EE87E3205C985EF...|      CMT|2013-01-07 23:54:15|   

In [ ]:
fareWorker.schema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)



In [ ]:
num_cols = [f.name for f in fareWorker.df.schema.fields if isinstance(f.dataType, t.NumericType)]
fareWorker.df.describe(num_cols).show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
from pyspark.sql.functions import col

class FareWorker:
    fare_schema = t.StructType([
        t.StructField(medallion, t.StringType(), False),
        t.StructField(hack_license, t.StringType(), False),
        t.StructField(vendor_id, t.StringType(), False),
        t.StructField(pickup_datetime, t.TimestampType(), False),
        t.StructField(payment_type, t.StringType(), False),
        t.StructField(fare_amount, t.FloatType(), False),
        t.StructField(surcharge, t.FloatType(), False),
        t.StructField(mta_tax, t.FloatType(), False),
        t.StructField(tip_amount, t.FloatType(), False),
        t.StructField(tolls_amount, t.FloatType(), False),
        t.StructField(total_amount, t.FloatType(), False),
    ])

    preprocess_filter_condition = (
            (col(fare_amount) > 0)
            & (col(surcharge) > 0)
            & (col(mta_tax) > 0)
            & (col(tip_amount) > 0)
            & (col(tolls_amount) > 0)
            & (col(total_amount) > 0))

    def __init__(self, spark: SparkSession):
        self.df = (spark.read
                   .csv(path_fare, self.fare_schema, header=True, ignoreTrailingWhiteSpace=True,
                        ignoreLeadingWhiteSpace=True, nullValue='null')
                   .filter(self.preprocess_filter_condition))

    def show(self):
        self.df.show()

    def count(self) -> int:
        return self.df.count()

    def schema(self):
        self.df.printSchema()

In [ ]:
fareWorker = FareWorker(spark)

In [ ]:
fareWorker.df.show()

+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+------------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+------------+
|F1C63842BC47D1C30...|9D0A6E44AF6C07CD7...|      VTS|2013-01-13 04:14:00|         CRD|       32.0|      0.5|    0.5|       6.5|         6.5|        46.0|
|ED9B6E969C35E1631...|1067C6EE7965AACBC...|      VTS|2013-01-13 03:46:00|         CRD|      107.5|      0.5|    0.5|      21.6|         4.8|       134.9|
|203F59092282F76DA...|711947488A628488F...|      VTS|2013-01-13 04:22:00|         CRD|       33.0|      0.5|    0.5|       6.7|         2.2|        42.9|
|AF654FCA5D10F5E1A...|47EB5C6F23BFD50C4...|      VTS|2013-01-13 04:46:00|   

In [ ]:
 num_cols = [f.name for f in fareWorker.df.schema.fields if isinstance(f.dataType, t.NumericType)]
fareWorker.df.describe(num_cols).show()

**Запити**

*Ляховський Даниїл*

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, avg, min, month, hour, count, lit, when

#Query 1: The average payment and average tip for each type of payment
def avg_income_by_payment_type(self):
        return (self.df
                .groupBy(col(payment_type))
                .agg(avg(total_amount).alias('avg_total_amount_by_payment_type'), avg(tip_amount).alias('avg_tip_amount_by_payment_type'))
                .orderBy('avg_total_amount_by_payment_type'))

#Query 2: Average duration and trip distance depending on the number of passengers
def avg_trip_by_passenger_count(self):
    return (self.df
            .groupBy(col(passenger_count))
            .agg((avg(col(trip_time_in_secs))/60).alias('avg_trip_duration_in_minutes'), avg(trip_distance).alias('avg_trip_distance_in_meters'))
            .orderBy('passenger_count'))

#Query 3: The number of trips according to each hour of the day
def trips_by_hour_of_day(self):
    return (self.df
            .withColumn('hour_of_day', hour(col(pickup_datetime)))
            .groupBy('hour_of_day')
            .count()
            .orderBy('hour_of_day'))

#Query 4: Average fare and number of trips per month
def avg_fare_and_trip_count_by_month(self, fare_df: DataFrame):
    return (self.df
            .join(fare_df,
                  (self.df['hack_license'] == fare_df['hack_license'])
                  & (self.df['pickup_datetime'] == fare_df['pickup_datetime']),
                  how='inner')
            .groupBy(month(self.df['pickup_datetime']).alias('pickup_month'))
            .agg(avg(fare_df['fare_amount']).alias('avg_fare_amount'),
                 count(lit(1)).alias('trip_count'))
            .orderBy('pickup_month'))
#Query 5:
def average_passenger_count(self, fare_df: DataFrame):
    window = Window.orderBy(col('hack_license'), col('pickup_datetime'))
    return (self.df
            .join(fare_df,
                  (self.df['hack_license'] == fare_df['hack_license'])
                  & (self.df['pickup_datetime'] == fare_df['pickup_datetime']),
                  how='inner')
            .withColumn('average_total_amount',
                        avg(fare_df['total_amount']).over(window))
            .withColumn('average_tip_amount',
                        avg(fare_df['tip_amount']).over(window))
            .select('medallion', 'hack_license', 'pickup_datetime', 'passenger_count', 'average_tip_amount', 'average_total_amount'))

#Query 6: Information about trips on December 31
def trips_info_on_december_31(self, fare_df: DataFrame):
    merged_df = self.df.alias("trip").join(fare_df.alias("fare"),
                                           (col("trip.medallion") == col("fare.medallion"))
                                           & (col("trip.hack_license") == col("fare.hack_license"))
                                           & (col("trip.pickup_datetime") == col("fare.pickup_datetime")),
                                           how='inner')
    window = Window.orderBy(col('trip.pickup_datetime')).partitionBy(col('trip.hack_license'))
    december_31_trips = merged_df.filter((col('trip.pickup_datetime').like('2013-12-31%')))
    result_df = (december_31_trips
                 .withColumn('total_amount_december_31', sum(col('fare.total_amount')).over(window))
                 .withColumn('tip_amount_december_31', sum(col('fare.tip_amount')).over(window))
                 .withColumn('trip_count_december_31', count(when(col('fare.total_amount').isNotNull(), 1)).over(window))
                 .select('trip.medallion', 'trip.hack_license', 'trip.vendor_id', 'trip.pickup_datetime',
                         'trip.dropoff_datetime', 'trip.trip_distance', 'trip.passenger_count',
                         'total_amount_december_31', 'tip_amount_december_31', 'trip_count_december_31',
                         'fare.fare_amount'))
    return result_df


FareWorker.avg_income_by_payment_type = avg_income_by_payment_type
TripWorker.avg_trip_by_passenger_count = avg_trip_by_passenger_count
TripWorker.trips_by_hour_of_day = trips_by_hour_of_day
TripWorker.avg_fare_and_trip_count_by_month = avg_fare_and_trip_count_by_month
TripWorker.average_passenger_count = average_passenger_count
TripWorker.trips_info_on_december_31 = trips_info_on_december_31

In [ ]:
res = fareWorker.avg_income_by_payment_type()
res.show()

+------------+--------------------------------+------------------------------+
|payment_type|avg_total_amount_by_payment_type|avg_tip_amount_by_payment_type|
+------------+--------------------------------+------------------------------+
|         UNK|               41.46988988587174|            5.9853078083869695|
|         CRD|               43.10881165649835|             6.771808027246697|
|         DIS|               47.32923111548791|             8.339230720813458|
|         CSH|              57.216788711919115|              9.50031631395964|
|         NOC|                67.1824008178711|            11.925600147247314|
+------------+--------------------------------+------------------------------+



In [ ]:
res = tripWorker.avg_trip_by_passenger_count()
res.show()

+---------------+----------------------------+---------------------------+
|passenger_count|avg_trip_duration_in_minutes|avg_trip_distance_in_meters|
+---------------+----------------------------+---------------------------+
|              1|           12.45363854943212|          2.864331676228944|
|              2|          13.221289035122933|          3.132683608506243|
|              3|          13.063846487562916|          2.995664093904258|
|              4|          13.292556630759751|         3.0704309022464362|
|              5|          12.767367898400495|         3.0059805461290425|
|              6|           12.71326754613647|          2.966485657952133|
|              7|           11.50701754385965|           4.07684211040798|
|              8|          16.612820512820512|          5.260000059237847|
|              9|          12.845098039215687|           3.40529416753527|
+---------------+----------------------------+---------------------------+



In [ ]:
res = tripWorker.trips_by_hour_of_day()
res.show(24)

+-----------+--------+
|hour_of_day|   count|
+-----------+--------+
|          0| 6859958|
|          1| 5058466|
|          2| 3751200|
|          3| 2732474|
|          4| 1992051|
|          5| 1700609|
|          6| 3542063|
|          7| 6237432|
|          8| 7735990|
|          9| 8011686|
|         10| 7770435|
|         11| 8018377|
|         12| 8449374|
|         13| 8359060|
|         14| 8616982|
|         15| 8188661|
|         16| 6860234|
|         17| 8369382|
|         18|10317800|
|         19|10795741|
|         20|10216129|
|         21|10009518|
|         22| 9707947|
|         23| 8597636|
+-----------+--------+



In [ ]:
res = tripWorker.avg_fare_and_trip_count_by_month(fareWorker.df)
res.show()

+------------+------------------+----------+
|pickup_month|   avg_fare_amount|trip_count|
+------------+------------------+----------+
|           1|29.325388236404976|    107280|
|           2|29.534481982189707|    106589|
|           3| 29.58458562334528|    124452|
|           4| 29.87224131218704|    125895|
|           5|30.280802025932452|    129522|
|           6|  30.1536204256245|    123433|
|           7|29.829746131253508|    111987|
|           8|29.878849114563323|    113278|
|           9|29.816513573721775|    126568|
|          10|29.798613519440696|    141870|
|          11|29.872976941160495|    117829|
|          12| 30.36345633734415|    119427|
+------------+------------------+----------+



In [ ]:
res = tripWorker.average_passenger_count(fareWorker.df)
res.show()

AnalysisException: ignored

In [ ]:
res = tripWorker.trips_info_on_december_31(fareWorker.df)
res.show()

+--------------------+--------------------+---------+-------------------+-------------------+-------------+---------------+------------------------+----------------------+----------------------+-----------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|   dropoff_datetime|trip_distance|passenger_count|total_amount_december_31|tip_amount_december_31|trip_count_december_31|fare_amount|
+--------------------+--------------------+---------+-------------------+-------------------+-------------+---------------+------------------------+----------------------+----------------------+-----------+
|BE530E79CB7E459DE...|0002555BBE359440D...|      CMT|2013-12-31 19:54:49|2013-12-31 20:21:24|          8.9|              1|      42.380001068115234|     7.050000190734863|                     1|       28.5|
|A8A7A370A7BAA9D3A...|0006C8F9279EFD18D...|      CMT|2013-12-31 02:48:42|2013-12-31 03:15:47|         11.7|              3|      50.189998626708984|     8.359999656677246| 

**========================================================================================================================================================================================================**

*Верхола Ярослав*

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, avg, min, month, hour, count, lit, when, rank, year, sum
from datetime import datetime

#Query 1: Average Trip Distance by Rate Code
def avg_trip_distance_by_rate_code(self):
    return (self.df
            .groupBy(col(rate_code))
            .agg(avg(trip_distance).alias('avg_trip_distance_by_rate_code'))
            .orderBy('rate_code'))

#Query 2: Number of Trips and Average Fare Amount by Vendor ID
def trips_and_avg_fare_by_vendor(self, fare_df: DataFrame):
    return (self.df
            .join(fare_df,
                  (self.df['medallion'] == fare_df['medallion'])
                  & (self.df['hack_license'] == fare_df['hack_license'])
                  & (self.df['pickup_datetime'] == fare_df['pickup_datetime']),
                  how='inner')
            .groupBy(self.df['vendor_id'])
            .agg(count(lit(1)).alias('trip_count'), avg(fare_df['fare_amount']).alias('avg_fare_amount'))
            .orderBy('vendor_id'))

#Query 3: Number of Trips and Average Tip Percentage by Payment Type
def trips_and_avg_tip_percentage_by_payment_type(self, fare_df: DataFrame):
    return (self.df
            .join(fare_df,
                  (self.df['medallion'] == fare_df['medallion'])
                  & (self.df['hack_license'] == fare_df['hack_license'])
                  & (self.df['pickup_datetime'] == fare_df['pickup_datetime']),
                  how='inner')
            .groupBy(col('payment_type'))
            .agg(count(lit(1)).alias('trip_count'),
                 (avg(fare_df['tip_amount']) / avg(fare_df['total_amount']) * 100).alias('avg_tip_percentage'))
            .orderBy('payment_type'))

#Query 4:What are the top 5 medallions with the highest total tip amounts for trips made by vendors 'CMT' and 'VTS' during the year 2013?
def top_medallions_with_highest_tips_window(self, fare_df: DataFrame):
    joined_data = self.df.join(fare_df, on=['medallion', 'hack_license', 'vendor_id', 'pickup_datetime'], how='inner')
    filtered_data = joined_data.filter((col('vendor_id').isin(['CMT', 'VTS'])) &
                                       (year(col('pickup_datetime')) == 2013) &
                                       (col('tip_amount') > 0))
    total_tip_amounts = (filtered_data
        .groupBy(col('medallion'))
        .agg(sum(col('tip_amount')).alias('total_tip_amount'))
    )
    window_spec = Window.orderBy(col('total_tip_amount').desc())
    ranked_medallions = (total_tip_amounts
        .withColumn('rank', rank().over(window_spec))
        .filter(col('rank') <= 5)
        .orderBy(col('rank'))
    )
    return ranked_medallions

#Query 5: What is the average trip distance for trips with a fare amount greater than $20, grouped by the rate code?
def avg_trip_distance_for_expensive_trips(self, fare_df: DataFrame):
    joined_data = self.df.join(fare_df, on=['medallion', 'hack_license', 'vendor_id', 'pickup_datetime'], how='inner')
    filtered_data = joined_data.filter((col('fare_amount') > 20) & (col('trip_distance') > 0))
    result_df = (filtered_data
        .groupBy(col('rate_code'))
        .agg(avg(col('trip_distance')).alias('avg_trip_distance'))
    )

    return result_df

#Query 6: For each vendor, what is the average tip amount and total number of trips per month during the year 2013?
def avg_tip_and_total_trips_per_month(self, fare_df: DataFrame):
    joined_data = self.df.join(fare_df, on=['medallion', 'hack_license', 'vendor_id', 'pickup_datetime'], how='inner')
    filtered_data = joined_data.filter((year(col('pickup_datetime')) == 2013) & (col('tip_amount') > 0))
    filtered_data = filtered_data.withColumn('pickup_month', month(col('pickup_datetime')))
    result_df = (filtered_data
        .groupBy(col('vendor_id'), col('pickup_month'))
        .agg(avg(col('tip_amount')).alias('avg_tip_amount'), count('*').alias('total_trips'))
    )
    return result_df



TripWorker.avg_trip_distance_by_rate_code = avg_trip_distance_by_rate_code
TripWorker.trips_and_avg_fare_by_vendor = trips_and_avg_fare_by_vendor
TripWorker.trips_and_avg_tip_percentage_by_payment_type = trips_and_avg_tip_percentage_by_payment_type
TripWorker.top_medallions_with_highest_tips_window = top_medallions_with_highest_tips_window
TripWorker.avg_trip_distance_for_expensive_trips = avg_trip_distance_for_expensive_trips
TripWorker.avg_tip_and_total_trips_per_month = avg_tip_and_total_trips_per_month


In [ ]:
res = tripWorker.avg_trip_distance_by_rate_code()
res.show()

+---------+------------------------------+
|rate_code|avg_trip_distance_by_rate_code|
+---------+------------------------------+
|        0|            3.1645385429058437|
|        1|             2.637870829841818|
|        2|            17.570905209133347|
|        3|               16.606677908804|
|        4|             8.900245971826527|
|        5|             10.66871056381719|
|        6|            3.5685297306486077|
|        7|            4.5296295591526565|
|        8|            1.7500000298023224|
|        9|             3.092307700560643|
|       15|                          28.5|
|       17|            0.6000000238418579|
|       28|              3.99999992052714|
|       65|             4.460000038146973|
|       77|             30.24999962002039|
|       79|            2.9999999403953552|
|      200|             4.099999904632568|
|      206|                          24.0|
|      210|            17.058333163460095|
|      221|            3.0999999046325684|
+---------+

In [ ]:
res = tripWorker.trips_and_avg_fare_by_vendor(fareWorker.df)
res.show()

+---------+----------+------------------+
|vendor_id|trip_count|   avg_fare_amount|
+---------+----------+------------------+
|      CMT|  86688059|12.207681141684876|
|      VTS|  86497032|12.493836446477616|
+---------+----------+------------------+



In [ ]:
res = tripWorker.trips_and_avg_tip_percentage_by_payment_type(fareWorker.df)
res.show()


+------------+----------+------------------+
|payment_type|trip_count|avg_tip_percentage|
+------------+----------+------------------+
|         CRD|   1444445|15.705226027588964|
|         CSH|       409|16.578058441851823|
|         DIS|        13| 17.61962010425423|
|         NOC|        22|18.436713374125073|
|         UNK|      3232|14.424854419071972|
+------------+----------+------------------+



In [ ]:
res = tripWorker.top_medallions_with_highest_tips_window(fareWorker.df)
res.show()

+--------------------+------------------+----+
|           medallion|  total_tip_amount|rank|
+--------------------+------------------+----+
|96B9D6925FCF4275E...| 4284.900004018098|   1|
|D29C5537B0FCD75F5...|3523.9199939370155|   2|
|C67E4F9B7A376BBC3...| 3418.989996790886|   3|
|A2025A43E119F430D...|3357.7600000053644|   4|
|6DBD2CAC39BDF995F...| 3331.349994972348|   5|
+--------------------+------------------+----+



In [ ]:
res = tripWorker.avg_trip_distance_for_expensive_trips(fareWorker.df)
res.show()

+---------+------------------+
|rate_code| avg_trip_distance|
+---------+------------------+
|        1|10.242936054156958|
|        4|21.290138125842425|
|        0|10.589534931404646|
|        2|17.303571479661123|
|        5|10.312500074505806|
|        3|15.439999930063884|
|        7| 7.900000095367432|
|      210|               9.0|
+---------+------------------+



In [ ]:
res = tripWorker.avg_tip_and_total_trips_per_month(fareWorker.df)
res.show()

+---------+------------+-----------------+-----------+
|vendor_id|pickup_month|   avg_tip_amount|total_trips|
+---------+------------+-----------------+-----------+
|      CMT|          12|7.153392068729986|      56523|
|      VTS|           2|6.256470951788356|      54961|
|      VTS|           1|6.200401433579257|      54679|
|      CMT|          10|6.984980552313909|      67619|
|      VTS|          10|7.078242171947447|      74251|
|      VTS|           8|6.281740179647902|      56954|
|      VTS|           9|6.709694920576118|      66573|
|      CMT|           1|6.855170233362057|      52601|
|      VTS|           7|6.275860394498265|      58793|
|      VTS|           6|6.360718369272998|      64758|
|      VTS|           5|6.379841589796838|      68176|
|      CMT|           8|7.007861125042788|      56324|
|      CMT|           2|6.897582603188723|      51626|
|      VTS|          12|7.261135058378472|      62904|
|      CMT|           9|6.991458787760324|      59995|
|      CMT

Запити Бобиляк Микола

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

    # Average tip amount by passenger count
def avg_tip_amount_by_passenger_count(self, fare_df: DataFrame):
      return (self.df
            .join(fare_df,
                  (self.df['hack_license'] == fare_df['hack_license'])
                  & (self.df['pickup_datetime'] == fare_df['pickup_datetime']),
                  how='inner')
            .groupBy(col(passenger_count))
            .agg(avg(total_amount).alias('avg_total_amount_by_passenger_count'))
            .orderBy('passenger_count'))

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, avg, min, month, hour, count, lit, when, to_date, sum, coalesce

    # Average tip amount by passenger count
def avg_tip_amount_by_passenger_count(self, fare_df: DataFrame):
      return (self.df
            .join(fare_df,
                  (self.df['hack_license'] == fare_df['hack_license'])
                  & (self.df['pickup_datetime'] == fare_df['pickup_datetime']),
                  how='inner')
            .groupBy(col(passenger_count))
            .agg(avg(tip_amount).alias('avg_tip_amount_by_passenger_count'))
            .orderBy('passenger_count'))

def avg_total_amount_by_distance(self, fare_df: DataFrame):
      return (self.df
            .join(fare_df,
                  (self.df['hack_license'] == fare_df['hack_license'])
                  & (self.df['pickup_datetime'] == fare_df['pickup_datetime']),
                  how='inner')
            .groupBy(col(trip_distance))
            .agg(avg(total_amount).alias('avg_total_amount_by_trip_distance'))
            .orderBy('trip_distance'))

#Avrage tips and tolls depends on fare amount
def avg_tips_and_tolls_amount_by_fare(self, fare_df: DataFrame):
    return (self.df
            .join(fare_df,
                  (self.df['hack_license'] == fare_df['hack_license'])
                  & (self.df['pickup_datetime'] == fare_df['pickup_datetime']),
                  how='inner')
            .groupBy(col(fare_amount))
            .agg(avg(tip_amount).alias('avg_tip_amount_depend_on_fare_amount'), avg(tolls_amount).alias('avg_tolls_amount_depend_on_fare_amount'))
            .orderBy(fare_amount))

#Average distance per month
def avg_distance_by_month(self):
    return (self.df
            .groupBy(month(self.df['pickup_datetime']).alias('pickup_month'))
            .agg(avg(trip_distance).alias('avg_trip_distance'))
            .orderBy('pickup_month'))


def amount_of_passengers_per_day(self):
        return (self.df
            .withColumn('pickup_date', to_date(col('pickup_datetime')))
            .groupBy('pickup_date')
            .agg(count('passenger_count').alias('passenger_count_per_day'))
            .orderBy('pickup_date')
)

def amount_of_passenger_for_payment_type(self, fare_df: DataFrame):
        return (self.df
                .join(fare_df,
                  (self.df['hack_license'] == fare_df['hack_license'])
                  & (self.df['pickup_datetime'] == fare_df['pickup_datetime']),
                  how='inner')
                .groupBy(col(payment_type))
                .agg(count('passenger_count').alias('passenger_count_for_payment_type'))
                .orderBy('payment_type'))


TripWorker.avg_tip_amount_by_passenger_count = avg_tip_amount_by_passenger_count
TripWorker.avg_total_amount_by_distance = avg_total_amount_by_distance
TripWorker.avg_distance_by_month = avg_distance_by_month
TripWorker.avg_tips_and_tolls_amount_by_fare = avg_tips_and_tolls_amount_by_fare
TripWorker.amount_of_passengers_per_day = amount_of_passengers_per_day
TripWorker.amount_of_passenger_for_payment_type = amount_of_passenger_for_payment_type

In [ ]:
res = tripWorker.avg_tip_amount_by_passenger_count(fareWorker.df)
res.show()

res = tripWorker.avg_total_amount_by_distance(fareWorker.df)
res.show()

res = tripWorker.avg_distance_by_month()
res.show()

res = tripWorker.avg_tips_and_tolls_amount_by_fare(fareWorker.df)
res.show()

res = tripWorker.amount_of_passengers_per_day()
res.show()